In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import torch
import torch.nn as nn
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from PIL import Image

c:\Users\Bastien Faivre\.conda\envs\myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataset import *
from utils import *
from model import UNET

In [3]:
IMAGE_HEIGHT = 400
IMAGE_WIDTH = 400
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 1e-4
TRAIN_IMAGE_DIR = "train_data/train_images/"
TRAIN_GROUNDTRUTH_DIR = "train_data/train_masks/"
VAL_IMAGE_DIR = "train_data/val_images/"
VAL_GROUNDTRUTH_DIR = "train_data/val_masks/"
BATCH_SIZE = 4
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 3

In [4]:
# define transformations
train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

val_transforms = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

In [5]:
# create model
model = UNET(in_channels=3, out_channels=1).to(DEVICE)
# define loss function, optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [6]:
# load data
train_loader, val_loader = get_loaders(
    TRAIN_IMAGE_DIR,
    TRAIN_GROUNDTRUTH_DIR,
    VAL_IMAGE_DIR,
    VAL_GROUNDTRUTH_DIR,
    BATCH_SIZE,
    train_transform,
    val_transforms,
    NUM_WORKERS,
    PIN_MEMORY
)

In [7]:
# compute F1 score
def f1_score(preds, target, threshold=0.5, eps=1e-7):
    preds = (preds > threshold).float()
    target = target.float()
    intersection = (preds * target).sum((1, 2))
    union = (preds + target).sum((1, 2))
    return ((2. * intersection + eps) / (union + eps)).mean()

In [8]:
def train_epoch(model, optimizer, criterion, train_loader, epoch, device):
    model.train()
    train_loss = 0
    train_f1 = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        predictions = model(data)
        loss = criterion(predictions, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_f1 += f1_score(predictions, target)
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tF1: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), f1_score(predictions, target)))
    train_loss /= len(train_loader)
    train_f1 /= len(train_loader)
    print('Train set: Average loss: {:.4f}\tAverage F1: {:.4f}'.format(train_loss, train_f1))
    return train_loss, train_f1

In [9]:
@torch.no_grad()
def validate(model, criterion, val_loader, device):
    model.eval()
    val_loss = 0
    val_f1 = 0
    for data, target in val_loader:
        data, target = data.to(device), target.float().unsqueeze(1).to(device)
        predictions = model(data)
        val_loss += criterion(predictions, target).item()
        val_f1 += f1_score(predictions, target)
    val_loss /= len(val_loader)
    val_f1 /= len(val_loader)
    print('Validation set: Average loss: {:.4f}\tAverage F1: {:.4f}'.format(val_loss, val_f1))
    return val_loss, val_f1

In [10]:
# train model

train_loss_history = []
train_f1_history = []
val_loss_history = []
val_f1_history = []

for epoch in range(NUM_EPOCHS):

    train_loss, train_f1 = train_epoch(model, optimizer, criterion, train_loader, epoch, DEVICE)

    train_loss_history.append(train_loss)
    train_f1_history.append(train_f1)

    val_loss, val_f1 = validate(model, criterion, val_loader, DEVICE)

    val_loss_history.append(val_loss)
    val_f1_history.append(val_f1)

    # save model
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer":optimizer.state_dict(),
    }
    save_checkpoint(checkpoint)

Train Epoch: 0 [0/80 (0%)]	Loss: 0.535924	F1: 0.683125
Train Epoch: 0 [40/80 (50%)]	Loss: 0.464567	F1: 0.758750
Train set: Average loss: 0.4649	Average F1: 0.6521
Validation set: Average loss: 0.5297	Average F1: 1.0000
=> Saving checkpoint
Train Epoch: 1 [0/80 (0%)]	Loss: 0.408914	F1: 0.803125
Train Epoch: 1 [40/80 (50%)]	Loss: 0.350653	F1: 0.890625
Train set: Average loss: 0.3577	Average F1: 0.8738
Validation set: Average loss: 0.3206	Average F1: 1.0000
=> Saving checkpoint
Train Epoch: 2 [0/80 (0%)]	Loss: 0.310879	F1: 0.940625
Train Epoch: 2 [40/80 (50%)]	Loss: 0.297009	F1: 0.866875
Train set: Average loss: 0.2947	Average F1: 0.9291
Validation set: Average loss: 0.2895	Average F1: 0.9097
=> Saving checkpoint
